<a href="https://colab.research.google.com/github/vagabond008/mids-266-Dipika-Jack-Ray/blob/main/W266_Final_Project_Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup

In [1]:
# install packages
#!pip install langdetect --quiet

In [2]:
# import libraries
import numpy as np
import pandas as pd
import re
from langdetect import detect
from google.colab import drive

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
# mount Colab to Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# verify data exists in Google Drive dir
!ls 'drive/My Drive/W266'

reddit_database_cleaned.csv  W266_Final_Project_Data_Cleanup.ipynb
reddit_database.csv	     W266_Final_Project_Pegasus_Ray.ipynb


# 2. Load Data

In [6]:
# load data
df = pd.read_csv('drive/My Drive/W266/reddit_database.csv')
df.head(3)

,created_date,created_timestamp,subreddit,title,id,author,author_created_utc,full_link,score,num_comments,num_crossposts,subreddit_subscribers,post
0,2010-02-10 22:06:17,1.265832e+09,analytics,YouTube's traffic data for music questioned,b0ih7,salvage,1.184143e+09,https://www.reddit.com/r/analytics/comments/b0ih7/youtubes_traffic_data_for_music_questioned/,3.0,0.0,0.0,NaN,NaN
1,2010-02-10 22:06:53,1.265832e+09,analytics,November Sees Number of U.S. Videos Viewed Online Surpass 30 Billion for First Time on Record [comScore],b0ihf,salvage,1.184143e+09,https://www.reddit.com/r/analytics/comments/b0ihf/november_sees_number_of_us_videos_viewed_online/,1.0,0.0,0.0,NaN,NaN
2,2010-02-11 19:47:22,1.265910e+09,analytics,So what do you guys all do related to analytics? Why the interest?,b0x63,xtom,1.227476e+09,https://www.reddit.com/r/analytics/comments/b0x63/so_what_do_you_guys_all_do_related_to_analytics/,7.0,4.0,0.0,NaN,"There's a lot of reasons to want to know all this stuff, so I figured I'd get to know the others that are on this subreddit.\n\nSo let's hear it: Webmasters? Coders? Marketers? Work for an analytics software company? You get the idea."


In [34]:
df = df[['title', 'post']]
df.head(5)

,title,post
0,YouTube's traffic data for music questioned,NaN
1,November Sees Number of U.S. Videos Viewed Online Surpass 30 Billion for First Time on Record [comScore],NaN
2,So what do you guys all do related to analytics? Why the interest?,"There's a lot of reasons to want to know all this stuff, so I figured I'd get to know the others that are on this subreddit.\n\nSo let's hear it: Webmasters? Coders? Marketers? Work for an analytics software company? You get the idea."
3,10 Web Analytics Tools For Tracking Your Visitors,NaN
4,Improving Your Sense of Site,NaN


In [35]:
#df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545427 entries, 0 to 545426
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   545427 non-null  object
 1   post    274209 non-null  object
dtypes: object(2)
memory usage: 8.3+ MB


# 3. Data Clensing

In [36]:
# drop NaNs from the title and post
df_cleaned = df.dropna(subset=['title', 'post'])
df_cleaned.head(5)

,title,post
2,So what do you guys all do related to analytics? Why the interest?,"There's a lot of reasons to want to know all this stuff, so I figured I'd get to know the others that are on this subreddit.\n\nSo let's hear it: Webmasters? Coders? Marketers? Work for an analytics software company? You get the idea."
5,"Google's Invasive, non-Anonymized Ad Targeting: A Quick Confirmation of previously suspected privacy issues","I'm cross posting this from /r/cyberlaw, hopefully you guys find it as interesting as I did(it deals with Google Analytics):\n\nSo quite awhile ago, I ordered a Papa John's pizza online. My job largely involves looking at ads that appear online, so afterwards I was quick to notice *I was getting a LOT* of Papa Johns ads (especially at night) being served through a Google owned company (DoubleClick media). Yesterday one of these ads popped up again on Youtube (a place that typically serves using the adwords program, not doubleclick), so I decided to copy the URL. \n\nFor those not in the advertising field: Making full use of Google's analytics tool means that certain information about the advertising campaign is leaked in the URL.\n\nSo let's break it apart: \n\n&gt;http://ad.doubleclick.net/click;h=(junk here);~sscs=?http://googleads.g.doubleclick.net/aclk?sa=l&amp;ai=(junk here)&amp;adurl=http://www.papajohns.com/index.shtm?utm_source=googlenetwork&amp;utm_medium=DisplayCPC&amp;utm_campaign=GoogleRemarketing\n\nFirst off, we see ~sscs: ~sscs is doubleclick's redirect variable. So rather than directly serving adwords ads, they overrode it to serve through doubleclick, then redirect through what would otherwise be an adwords link(http://googleads.g.doubleclick.net). This is tighter integration than is generally seen with adwords/doubleclick.\n\n* The interesting part is the end variables utm_source=**googlenetwork**&amp;utm_medium=**DisplayCPC**&amp;utm_campaign=**GoogleRemarketing**\n\n* DisplayCPC/googlenetwork - Confirmation that doubleclick is now more finely integrated with adwords.\n\n* ""GoogleRemarketing"", huh? Let's take a look at the definition for ""Remarketing""\n\n&gt;Using past campaign information to target a particular message to an audience.\n\nWhile in the past behavioral targetting has largely been based on the sum of your use, this is an interesting(though no doubt more widespread than is known) change in that; explicitly targeting old customers though a *massive* network of sites.\n\n-----------------------------------\n\nJust thought I'd put this out there. I'm sure it's not new to a lot of people, but at least to me it was interesting to see concepts like this actually put into practice on such a large scale. \n\n-----------------------------\n\nPS: I did a quick survey across several thousand domains, and for the record: right now, the most common external resource locations on the internet are(Google owned is bolded):\n\n**www.google-analytics.com**\n\n**pagead2.googlesyndication.com**\n\n**googleads.g.doubleclick.net**\n\nedge.quantserve.com\n\n**ad.doubleclick.net**\n\n**www.youtube.com**\n\nb.scorecardresearch.com\n\ns0.2mdn.net\n\ndg.specificclick.net\n\nview.atdmt.com\n\n**www.google.com**\n\n**ajax.googleapis.com**\n\n**partner.googleadservices.com**\n\nThat's a lot of data."
62,"DotCed - Functional Web Analytics - Tagging, Reporting, Analysis, and Strategy - www.dotced.com","DotCed,a Functional Analytics Consultant, offering Google Analytics Tagging, Reporting, Analysis, Strategy, SEO Auditing, and SEM Optimization. Call 919-404-9233 for a 15 min consultation."
64,Program Details - Data Analytics Course,Here is the program details of the data analytics certification course at the Academy for Decision Science Ahmedabad.
65,potential job in web analytics... need to analyze some data. what are they looking for?,"i decided grad school (physics) was not for me and i am branching out into the job market. a web analytics place is interested in me (and i'm interested in any kind o

In [37]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274209 entries, 2 to 545425
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   274209 non-null  object
 1   post    274209 non-null  object
dtypes: object(2)
memory usage: 6.3+ MB


In [38]:
df_nulls = df['post'].isnull()
df_cleaned[df_nulls]

<ipython-input-38-15d9d89a55bb>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_cleaned[df_nulls]


,title,post


In [39]:
# pre-inspection
df_cleaned[:5]

,title,post
2,So what do you guys all do related to analytics? Why the interest?,"There's a lot of reasons to want to know all this stuff, so I figured I'd get to know the others that are on this subreddit.\n\nSo let's hear it: Webmasters? Coders? Marketers? Work for an analytics software company? You get the idea."
5,"Google's Invasive, non-Anonymized Ad Targeting: A Quick Confirmation of previously suspected privacy issues","I'm cross posting this from /r/cyberlaw, hopefully you guys find it as interesting as I did(it deals with Google Analytics):\n\nSo quite awhile ago, I ordered a Papa John's pizza online. My job largely involves looking at ads that appear online, so afterwards I was quick to notice *I was getting a LOT* of Papa Johns ads (especially at night) being served through a Google owned company (DoubleClick media). Yesterday one of these ads popped up again on Youtube (a place that typically serves using the adwords program, not doubleclick), so I decided to copy the URL. \n\nFor those not in the advertising field: Making full use of Google's analytics tool means that certain information about the advertising campaign is leaked in the URL.\n\nSo let's break it apart: \n\n&gt;http://ad.doubleclick.net/click;h=(junk here);~sscs=?http://googleads.g.doubleclick.net/aclk?sa=l&amp;ai=(junk here)&amp;adurl=http://www.papajohns.com/index.shtm?utm_source=googlenetwork&amp;utm_medium=DisplayCPC&amp;utm_campaign=GoogleRemarketing\n\nFirst off, we see ~sscs: ~sscs is doubleclick's redirect variable. So rather than directly serving adwords ads, they overrode it to serve through doubleclick, then redirect through what would otherwise be an adwords link(http://googleads.g.doubleclick.net). This is tighter integration than is generally seen with adwords/doubleclick.\n\n* The interesting part is the end variables utm_source=**googlenetwork**&amp;utm_medium=**DisplayCPC**&amp;utm_campaign=**GoogleRemarketing**\n\n* DisplayCPC/googlenetwork - Confirmation that doubleclick is now more finely integrated with adwords.\n\n* ""GoogleRemarketing"", huh? Let's take a look at the definition for ""Remarketing""\n\n&gt;Using past campaign information to target a particular message to an audience.\n\nWhile in the past behavioral targetting has largely been based on the sum of your use, this is an interesting(though no doubt more widespread than is known) change in that; explicitly targeting old customers though a *massive* network of sites.\n\n-----------------------------------\n\nJust thought I'd put this out there. I'm sure it's not new to a lot of people, but at least to me it was interesting to see concepts like this actually put into practice on such a large scale. \n\n-----------------------------\n\nPS: I did a quick survey across several thousand domains, and for the record: right now, the most common external resource locations on the internet are(Google owned is bolded):\n\n**www.google-analytics.com**\n\n**pagead2.googlesyndication.com**\n\n**googleads.g.doubleclick.net**\n\nedge.quantserve.com\n\n**ad.doubleclick.net**\n\n**www.youtube.com**\n\nb.scorecardresearch.com\n\ns0.2mdn.net\n\ndg.specificclick.net\n\nview.atdmt.com\n\n**www.google.com**\n\n**ajax.googleapis.com**\n\n**partner.googleadservices.com**\n\nThat's a lot of data."
62,"DotCed - Functional Web Analytics - Tagging, Reporting, Analysis, and Strategy - www.dotced.com","DotCed,a Functional Analytics Consultant, offering Google Analytics Tagging, Reporting, Analysis, Strategy, SEO Auditing, and SEM Optimization. Call 919-404-9233 for a 15 min consultation."
64,Program Details - Data Analytics Course,Here is the program details of the data analytics certification course at the Academy for Decision Science Ahmedabad.
65,potential job in web analytics... need to analyze some data. what are they looking for?,"i decided grad school (physics) was not for me and i am branching out into the job market. a web analytics place is interested in me (and i'm interested in any kind o

In [40]:
# English lang detection
# def is_english(text):
#     try:
#         return detect(text) == 'en'
#     except:
#         return False

# text normalization
def text_normalization(text):
  # # check if text is English
  # if not is_english(text):
  #   return np.nan

  # Remove HTML tags
  text = re.sub(r'<.*?>', '', text)
  # Remove URLs
  text = re.sub(r'https?://\S+|www\.\S+', '', text)
  # Remove newlines
  text = re.sub(r'\n', ' ', text)
  # Remove tabs
  text = re.sub(r'\t', ' ', text)
  # Remove removes numeric sequences
  text = re.sub(r'\d+', '', text)
  # Remove punctuation
  text = re.sub(r'[^\w\s]', '', text)
  # Remove punctuation
  text = re.sub(r'[()]', '', text)
  # convert to lowercase
  text = text.lower()
  # remove punctuation
  text = re.sub(r'[^\w\s]', '', text)
  # remove extra spaces
  text = re.sub(r' +', ' ', text.strip())

  return text

# apply text normalization
df_cleaned['post'] = df_cleaned['post'].apply(text_normalization)
df_cleaned['title'] = df_cleaned['title'].apply(text_normalization)
df_cleaned = df_cleaned.reset_index(drop=True)

<ipython-input-40-80707ff6a66b>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['post'] = df_cleaned['post'].apply(text_normalization)
<ipython-input-40-80707ff6a66b>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['title'] = df_cleaned['title'].apply(text_normalization)


In [49]:
# post-inspection
df_cleaned[:5]

,title,post
0,so what do you guys all do related to analytics why the interest,theres a lot of reasons to want to know all this stuff so i figured id get to know the others that are on this subreddit so lets hear it webmasters coders marketers work for an analytics software company you get the idea
1,googles invasive nonanonymized ad targeting a quick confirmation of previously suspected privacy issues,im cross posting this from rcyberlaw hopefully you guys find it as interesting as i didit deals with google analytics so quite awhile ago i ordered a papa johns pizza online my job largely involves looking at ads that appear online so afterwards i was quick to notice i was getting a lot of papa johns ads especially at night being served through a google owned company doubleclick media yesterday one of these ads popped up again on youtube a place that typically serves using the adwords program not doubleclick so i decided to copy the url for those not in the advertising field making full use of googles analytics tool means that certain information about the advertising campaign is leaked in the url so lets break it apart gt heresscs hereampadurl first off we see sscs sscs is doubleclicks redirect variable so rather than directly serving adwords ads they overrode it to serve through doubleclick then redirect through what would otherwise be an adwords link this is tighter integration than is generally seen with adwordsdoubleclick the interesting part is the end variables utm_sourcegooglenetworkamputm_mediumdisplaycpcamputm_campaigngoogleremarketing displaycpcgooglenetwork confirmation that doubleclick is now more finely integrated with adwords googleremarketing huh lets take a look at the definition for remarketing gtusing past campaign information to target a particular message to an audience while in the past behavioral targetting has largely been based on the sum of your use this is an interestingthough no doubt more widespread than is known change in that explicitly targeting old customers though a massive network of sites just thought id put this out there im sure its not new to a lot of people but at least to me it was interesting to see concepts like this actually put into practice on such a large scale ps i did a quick survey across several thousand domains and for the record right now the most common external resource locations on the internet aregoogle owned is bolded pageadgooglesyndicationcom googleadsgdoubleclicknet edgequantservecom addoubleclicknet bscorecardresearchcom smdnnet dgspecificclicknet viewatdmtcom ajaxgoogleapiscom partnergoogleadservicescom thats a lot of data
2,dotced functional web analytics tagging reporting analysis and strategy,dotceda functional analytics consultant offering google analytics tagging reporting analysis strategy seo auditing and sem optimization call for a min consultation
3,program details data analytics course,here is the program details of the data analytics certification course at the academy for decision science ahmedabad
4,potential job in web analytics need to analyze some data what are they looking for,i decided grad school physics was not for me and i am branching out into the job market a web analytics place is interested in me and im interested in any kind of data analysis the exercise is to use a comparison of three or more months of data to prepare a to slide powerpoint presentation of any significant information about site visitors what they are doing how they arrive at our site that we could use to improve site performance as an acquisition source he said i should tell a story this is a field i am unfamiliar with so im looking for any basic tips common pitfalls and expectations thanks i am quite familiar with data analysis in general


In [47]:
df_cleaned.tail(5)

,title,post
274204,help interpretting lmer model output,hello i am wonder how the following output would be interpreted i ran a piecewise linear mixed effect model with fixed effect time predictors each time variable represents a segment of time the full length of time is units time is coded c etc time is cetc and time is coded c the dependent variable is an logx transformation to my knowledge the random effects are the betweengroup effects and the fixed effects are withingroup with group being soccodef im not sure how to interpret the effects though help linear mixed model fit by maximum likelihood ttests use satterthwaites method lmermodlmertest formula logpint time itime time time soccodef data onet aic bic loglik deviance dfresid scaled residuals min q median q max random effects groups name variance stddev soccodef intercept residual number of obs groups soccodef fixed effects estimate std error df t value prgtt intercept e e e lt e time e e e lt e itime e e e lt e time e e e e time e e e signif codes correlation of fixed effects intr time it time time itime time time
274205,medical stats book with r,can anybody recommend me a book with medical statistics with r thanks a lot
274206,markov chains with unequal sequence lengths,im trying to build a simple markov chain i have data from therapy notes where the therapist selects the overall topic of the session out a list of possible topics the problem is that not every therapist tags their session topic consistently and all clients have a different number of sessions im trying to build a simple markov chain to get probabilities of topic transition between sessions but as you can see the data are complicated i was wondering if anyone has encountered a situation where there were uneven sequence lengths per observationcase and how did you go about building a markov chain in this case thanks
274207,view all available rcppplugins,how do i view all available rcppplugins thanks
274208,print only loadings in factanal,hi everybody i am currently doing a factor analysis using the factanalcommand since i want to export the results to excel i want to basically get a table where all the loadings of all the items for all the factors are listed but every time i print the loadings it also prints the ss loadings the proportion variance and the cumulative variance for all the factors right below the loadings for the items i had the same problem when i did a principal component analysis and solved it there by only printing and exporting the weights instead of the loadings but factanal doesnt seem to have such a component does anybody know how i could fix that


In [44]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274209 entries, 0 to 274208
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   274209 non-null  object
 1   post    274209 non-null  object
dtypes: object(2)
memory usage: 4.2+ MB


In [45]:
# inspect NaNs
df_cleaned[df_cleaned.isna().any(axis=1)]

,title,post


In [46]:
# create clean reddit database file
df_cleaned.to_csv('drive/My Drive/W266/reddit_database_cleaned.csv', index=False)